In [118]:
import os 
import sys
import pygwalker as pyg
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np

In [119]:

#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
#Linux
#os.chdir('/home/kwnabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models
finbert_models = config.finbert_models

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
Model_Folder = config.Sentiment_models
Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2


/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [120]:
df = pd.read_csv(f"{finbert_models}/{Body}_{Model}_finbert model.csv")
df_2 = pd.read_csv(f"{finbert_models}/{Body_2}_{Model_2}_finbert model.csv")

print(df.head())
print(df_2.head())
print(df_2.columns)


   Unnamed: 0        date                                              title  \
0           0  1998-11-12          Modernizing financial services regulation   
1           1  1998-11-06                   Examining community reinvestment   
2           2  1998-11-05  The structure of the international financial s...   
3           3  1998-10-29           Countdown to Y2K: An economic assessment   
4           4  1998-10-27  The future of the financial services sector an...   

                              type  \
0       Governor Laurence H. Meyer   
1      Governor Edward M. Gramlich   
2          Chairman Alan Greenspan   
3   Governor Edward W. Kelley, Jr.   
4  Governor Roger W. Ferguson, Jr.   

                                             segment sentiment      tone  
0  modernizing financial services regulation  it ...  negative -0.010526  
1  examining community reinvestment  the federal ...  negative -0.017647  
2   this afternoon i intend to address a subject ...   neutral -0

In [121]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'neutral' : 0, 'negative' : -1})
df_2['sentiment'] = df_2['sentiment'].replace({'positive': 1, 'neutral' : 0, 'negative' : -1})

KeyError: 'sentiment'

In [ ]:
print(df.columns)
print(df_2.columns)
print(df.head())

In [ ]:
df_group = df.copy(deep=True)
df_group = df_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment'])

df_group['monthly'] = df_group['date'].str[0:7]
#df_group['monthly'] = df_group['date']

print(df_group.columns)
df_group = df_group[['monthly', 'sentiment']]
df_group = df_group.groupby('monthly', as_index = False, dropna=True).mean()

df_2_group = df_2.copy(deep=True)
df_2_group = df_2_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment'])
df_2_group['date'] = df_2_group['date'].astype(str)

#df_2_group['monthly'] = df_2_group['date']
df_2_group['monthly'] = df_2_group['date'].str[0:7]

df_2_group = df_2_group[['monthly', 'sentiment']]
df_2_group = df_2_group.groupby('monthly', as_index = False, dropna=True).mean()

df_group = df_group.rename(columns = {'sentiment' : f'{Body}_sentiment'})
df_2_group = df_2_group.rename(columns = {'sentiment' : f'{Body_2}_sentiment'})

print(df_group.head())
print(df_2_group.head())

df_merged = df_group.merge(df_2_group, on='monthly', how='inner')
df_merged.fillna(0, inplace=True)
print(df_merged.columns)

In [ ]:

df_merged['net_diff'] = df_merged['ECB_sentiment'] - df_merged['Fed_sentiment']

In [ ]:
funds = pd.read_excel('/Users/kylenabors/Documents/Database/Other Data/FedFundsRate.xlsx', sheet_name='Monthly')
print(funds.head())
energy = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/Energy Prices/US Energy CPI.csv')
print(energy.head())

In [ ]:
funds['Diff'] = funds['FEDFUNDS'].diff()


In [ ]:
df_merged.to_csv(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv", index=False)

In [ ]:
#gwalker = pyg.walk(df_merged)

In [ ]:
print(df_merged.describe())

In [ ]:


# Create a list of x-axis tick positions
num_ticks = 60  # Adjust this number as needed
x_tick_positions = df_merged['monthly'].iloc[::len(df_merged) // num_ticks].tolist()

y = 0
fig, axs = plt.subplots(3, 1, figsize=(40, 40))
axs[0].plot(df_merged['monthly'], df_merged[f'{Body}_sentiment'], color='blue', label='Net Sentiment')
axs[1].plot(df_merged['monthly'], df_merged[f'{Body_2}_sentiment'], color='red', label='Net Sentiment')
axs[0].hlines(y, df_merged['monthly'].min(), df_merged['monthly'].max(), color='black', label='Zero')
axs[1].hlines(y, df_merged['monthly'].min(), df_merged['monthly'].max(), color='black', label='Zero')

axs[2].plot(df_merged['monthly'], df_merged['net_diff'], color='green', label='Net Sentiment')
axs[2].hlines(y, df_merged['monthly'].min(), df_merged['monthly'].max(), color='black', label='Zero')
# Set x-axis ticks for both subplots
axs[0].set_xticks(x_tick_positions)
axs[1].set_xticks(x_tick_positions)
axs[2].set_xticks(x_tick_positions)

axs[0].tick_params(axis='x', rotation=45, labelsize=15)
axs[1].tick_params(axis='x', rotation=45, labelsize=15)
axs[2].tick_params(axis='x', rotation=45, labelsize=15)

axs[0].set_yticks(np.arange(-0.8, 0.6, .1))
axs[1].set_yticks(np.arange(-0.8, 0.6, .1))
axs[2].set_yticks(np.arange(-0.8, 0.6, .1))

axs[0].set_ylabel(f'{Body} Sentiment', fontsize=20)
axs[1].set_ylabel(f'{Body_2} Sentiment', fontsize=20)
axs[2].set_ylabel(f'Net Sentiment', fontsize=20)

plt.show()


In [ ]:
print(len(df_merged))
df_merged.to_csv(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv", index=False)
print(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv")